In [1]:
import sys
sys.path.append("..")
from Functions.UNSW_DF import *

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
#from keras_visualizer import visualizer
#from ann_visualizer.visualize import ann_viz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
x_train, x_test, y_train, y_test = DF_XY()

( 1 ) Reading Preprocessed CSV files..
	 Training dataset loaded..
	 Testing dataset loaded..

( 2 ) Loading done, splitting into X and Y..
	 ( 2.1 ) x_train Shape:  	 (175341, 53)
	 ( 2.2 ) y_train Shape:  	 (175341,)
	 ( 2.3 ) x_test Shape:  	 (82332, 53)
	 ( 2.4 ) y_test Shape:  	 (82332,)
( 3 ) Done!
PS! Import with: x_train, x_test, y_train, y_test = XY_import()


In [3]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=53, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [4]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
ann_accuracy = []
ann_precision = []
ann_recall = []
ann_f1 = []
ann_epochs = []
ann_no_of_layers = []

for i in range(1, 30):
    print(f"\n## ---------- EPOCH {i} ----------- ##\n")
    # fit the keras model on the dataset
    model.fit(x_train, y_train, epochs=i, batch_size=100)
    
    # predict probabilities for test set
    yhat_probs = model.predict(x_test, verbose=0)
    # predict crisp classes for test set
    yhat_classes = model.predict_classes(x_test, verbose=0)
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    ann_accuracy.append(accuracy)
    print('Accuracy: %f' % accuracy)

    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    ann_precision.append(precision)
    print('Precision: %f' % precision)

    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    ann_recall.append(recall)
    print('Recall: %f' % recall)

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    ann_f1.append(f1)
    print('F1 score: %f' % f1)
    
    ann_no_of_layers.append(len(model.layers))
    ann_epochs.append(i)
    


## ---------- EPOCH 1 ----------- ##

1754/1754 [==============================] - 1s 703us/step - loss: 0.1684 - accuracy: 0.9282
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Accuracy: 0.826957
Precision: 0.769269
Recall: 0.979507
F1 score: 0.861751

## ---------- EPOCH 2 ----------- ##

Epoch 1/2
1754/1754 [==============================] - 1s 673us/step - loss: 0.1263 - accuracy: 0.9387
Epoch 2/2
1754/1754 [==============================] - 1s 663us/step - loss: 0.1224 - accuracy: 0.9401
Accuracy: 0.848346
Precision: 0.795739
Recall: 0.974786
F1 score: 0.876210

## ---------- EPOCH 3 ----------- ##

Epoch 1/3
1754/1754 [==============================] - 2s 866us/step - loss: 0.1207 - accuracy: 0.94

In [6]:
# evaluate the keras model for training and testing data
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy for training\t: %.2f' % (train_acc*100))
print('Accuracy for testing\t: %.2f' % (test_acc*100))

Accuracy for training	: 95.06
Accuracy for testing	: 87.14


In [8]:
# dictionary of lists 
dict = {#'epochs': ann_epochs, 
        #'number_of_layers': ann_no_of_layers, 
        'accuracy': ann_accuracy, 
        'precision': ann_precision, 
        'f1': ann_f1, 
        'recall': ann_recall
        } 
    
df = pd.DataFrame(dict, index=ann_epochs)

df

,accuracy,precision,f1,recall
1,0.826957,0.769269,0.861751,0.979507
2,0.848346,0.795739,0.876210,0.974786
3,0.833431,0.770018,0.867982,0.994507
4,0.853204,0.798449,0.880372,0.981029
5,0.862107,0.812439,0.886139,0.974543
6,0.862666,0.810725,0.887024,0.979176
7,0.874702,0.831315,0.894923,0.969073
8,0.856508,0.801183,0.883002,0.983433
9,0.862945,0.812058,0.887031,0.977257
10,0.851747,0.797265,0.879208,0.979926


In [14]:
#UNSW_barplot(data=df, to_range=29 ,x_label="epochs", y_label="Score", title="ANN", x_size=30, y_size=8)

In [ ]:
#ann_viz(model, title="Neural Network")

In [ ]:
#visualizer(model, format='png', view=True)